In [0]:
from pyspark.sql import functions as F

In [0]:
# =========================
# Configurações
# =========================
storage_account = "storagevinicius"
container = "datalake"

landing_path = f"abfss://{container}@{storage_account}.dfs.core.windows.net/landing/CPIAUCSL.csv"
bronze_path  = f"abfss://{container}@{storage_account}.dfs.core.windows.net/bronze/cpiaucsl_delta"

In [0]:
try:
    df_raw = (
        spark.read
        .option("header", True)
        .option("inferSchema", True)
        .csv(landing_path)
    )
    print("Ingestão realizada com sucesso")
except:
    print ("Erro na ingestão")

In [0]:
# Metadados técnicos (permitido no Bronze)
df_bronze = (
    df_raw
    .withColumn("_ingestion_time", F.current_timestamp())
    .withColumn("_source_file", F.input_file_name())
)

#padronizar nomes das colunas
for c in df_bronze.columns:
    df_bronze = df_bronze.withColumnRenamed(c, c.strip().lower().replace(" ", "_"))

df_bronze.printSchema()
display(df_bronze.limit(10))

In [0]:
(
    df_bronze.write
    .format("delta")
    .mode("overwrite")
    .save(bronze_path)
)

print("Bronze salvo em Delta em:", bronze_path)

In [0]:
df_check = spark.read.format("delta").load(bronze_path)
print("Linhas no Bronze:", df_check.count())
display(df_check.limit(10))